# Azure Machine Learning for Computer Vision Package 

A large number of problems in the computer vision domain can be solved using image classification approaches. These include building models which answer questions such as, "Is an OBJECT present in the image?" (where OBJECT could for example be "dog", "car", "ship", etc.) as well as more complex questions, like "What class of eye disease severity is evinced by this patient's retinal scan?"

This notebook shows how the Azure Machine Learning for Computer Vision Package (CVTK) can be used to train, test, and deploy an **image classification** model. For more details, please check the API doc. Currently, [CNTK](https://www.microsoft.com/en-us/cognitive-toolkit/) is used as the deep learning framework, training is performed locally on a GPU powered machine such as the ([Data Science VM](https://azuremarketplace.microsoft.com/en-us/marketplace/apps/microsoft-ads.dsvm-deep-learning?tab=Overview)), and deployment uses the Azure ML Operationalization CLI.

The following steps are performed:
1. Dataset Creation
2. Image Visualization and annotation
3. Image Augmentation
4. DNN Model Definition
5. Classifier Training
6. Evaluation and Visualization
7. Webservice Deployment
8. Webservice Load Testing

### Example Dataset

As running example we use a dataset consisting of 63 tableware images, each labeled as belonging to one of four different classes (bowl, cup, cutlery, plate). We kept the number of images small so that this tutorial can be executed quickly; in practice at least 100 images per class should be provided. All images are located at *"../sample_data/imgs_recycling/"*, in subdirectories called "bowl", "cup", "cutlery", and "plate".

<img src="misc/recycling_examples.jpg" bowl width=800>


### Storage Context
The storage context is used to determine where various output files such as augmented images or DNN model files will be stored (see the StorageContext documentation for more details). Normally, the storage content does not need to be set explicitely. However, when using the AML Workbench, to avoid its 25 MB limit on the project size, we are setting the cvtk outputs directory to point to a location outside the AML project ("../../../../cvtk_output"). Make sure to remove the "cvtk_output" directory once it is no longer needed.

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import json, numpy as np, os, timeit 
from azureml.logging import get_azureml_logger
from imgaug import augmenters
from IPython.display import display
from sklearn import svm
from cvtk import ClassificationDataset, CNTKTLModel, Context, Splitter, StorageContext
from cvtk.augmentation import augment_dataset
from cvtk.core.classifier import ScikitClassifier
from cvtk.evaluation import ClassificationEvaluation, graph_roc_curve, graph_pr_curve, graph_confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

# Disable printing of logging messages
from azuremltkbase.logging import ToolkitLogger
ToolkitLogger.getInstance().setEnabled(False)

# Set storage context.
out_root_path = "../../../cvtk_output"
Context.create(outputs_path=out_root_path, persistent_path=out_root_path, temp_path=out_root_path)

### Step 1: Dataset Creation

The recommended way to generate a Dataset object in CVTK is by providing the root directory of the images on the local disk. This directory has to follow the same general structure as the tableware dataset, ie. contain sub-directories with the actual images:
- root
    - label1
    - label2
    - ...
    - labeln
  
Training an image classification model for a different dataset is therefore as easy as changing the root path `dataset_location` in the code below to point at different images.

In [ ]:
# Root image directory 
dataset_location = os.path.abspath(os.path.join(os.getcwd(), "../sample_data/imgs_recycling"))

dataset_name = 'recycling'
dataset = ClassificationDataset.create_from_dir(dataset_name, dataset_location)
print("Dataset consists of {} images with {} labels.".format(len(dataset.images), len(dataset.labels)))
print("Select information for image 2: name={}, label={}, unique id={}.".format(
    dataset.images[2].name, dataset.images[2]._labels[0].name, dataset.images[2]._storage_id))

### Step 2: Image visualization and annotation

The widget below can be used to visualize the images in the dataset object, and to correct some of the labels if needed. In case you see a "Widget Javascript not detected" error, this can often be solved by running the command: 
  `jupyter nbextension enable --py --sys-prefix widgetsnbextension`

In [ ]:
from ui_utils.ui_annotation import AnnotationUI
annotation_ui = AnnotationUI(dataset, Context.get_global_context())
display(annotation_ui.ui)

### Step 3: Image Augmentation

The augmentation module provides functionality to augment a dataset object using all the transformations described in the [imgaug](https://github.com/aleju/imgaug) library. Image transformations can be grouped in a single pipeline, in which case all transformations in the pipeline are applied simultaniously to each image. If you would like to apply different augmentation steps separately, or in any different manner, you can define multiple pipelines and pass them to the *augment_dataset* function. For more details and examples of image augmentation, see the [imgaug documentation](https://github.com/aleju/imgaug).

Adding augmented images to the training set is especially beneficial for small datasets. However, since it slows down DNN training due to the increased number of training images, our recommendation is to start experimentation without augmentation.

In [ ]:
# Split the dataset into train and test  
train_set_orig, test_set = dataset.split(train_size = 0.66, stratify = "label", random_state = 0)
print("Number of training images = {}, test images = {}.".format(train_set_orig.size(), test_set.size()))

In [ ]:
augment_train_set = False

if augment_train_set:
    aug_sequence = augmenters.Sequential([
            augmenters.Fliplr(0.5),             # horizontally flip 50% of all images
            augmenters.Crop(percent=(0, 0.1)),  # crop images by 0-10% of their height/width
        ])
    train_set = augment_dataset(train_set_orig, [aug_sequence])
    print("Number of original training images = {}, with augmented images included = {}.".format(train_set_orig.size(), train_set.size()))
else:
    train_set = train_set_orig  

### Step 4: DNN Model Definition

Six different per-trained Deep Neural Network models are supported in CVTK: AlexNet, Resnet-18, Resnet-34, and Resnet-50, Resnet-101, and Resnet-152. These DNNs can be used either as classifier, or as featurizer (see step 5). More information about the networks can be found [here](https://github.com/Microsoft/CNTK/blob/master/PretrainedModels/Image.md), and a basic introduction to Transfer Learning is [here](https://blog.slavv.com/a-gentle-intro-to-transfer-learning-2c0b674375a0).

CVTK comes with default parametes (224x224 pixel resolution and Resnet-18 DNN) which were selected to work well on a wide variety of tasks. Accuracy can often be improved by eg. increasing the image resolution to 500x500 pixels, and/or selecting a deeper model (Resnet-50), however this comes at a significant increase in training time. See the "How to improve accuracy" section in the Appendix for more detail, and why the minibatch-size and the learning rate need to be updated.

In [ ]:
# CVTK default parameters (224 x 224 pixels resolution, Resnet-18)
lr_per_mb = [0.05]*7 + [0.005]*7 +  [0.0005]
mb_size = 32
input_resoluton = 224
base_model_name = "ResNet18_ImageNet_CNTK"

# CVTK suggested parameters for 500 x 500 pixels resolution, Resnet-50
# (see in the Appendix "How to improve accuracy", last row in table)
# lr_per_mb   = [0.01] * 7 + [0.001] * 7 + [0.0001]
# mb_size    = 8
# input_resoluton = 500
# base_model_name = "ResNet50_ImageNet_CNTK"

# Initialize model
dnn_model = CNTKTLModel(train_set.labels,
                       base_model_name=base_model_name,
                       image_dims = (3, input_resoluton, input_resoluton))

### Step 5: Classifier Training

The pre-trained DNN from the last section can be used in two ways:
  - DNN refinement: This trains the DNN to directly perform the classification. While DNN training is slow, it typically leads to the best results since all network weights can be improved during training to give best accuracy.
  - DNN featurization: This runs the DNN as-is to obtain a lower-dimensional representation of an image (512, 2048, or 4096 floats), which is then used as input to train a separate classifier. Since the DNN is kept unchanged, this approach is much faster compared to DNN refinement, however accuracy is not as good. Nevertheless, training an external classifier such as a linear SVM (shown below) can provide a strong baseline, and help with understanding the feasibility of a problem.
  
#### Tensorboard Logging
Tensorboard can be used to visualize the training progress. To activate, 
1. Add the parameter `tensorboard_logdir=PATH` as shown below. 
2. Then start the tensorboard client by running in a new console the command `tensorboard --logdir=PATH` 
3. Open a web-browser as instructed by tensorboard (default is localhost:6006). 

`PATH` is the path to which you will log your files. Please note, you will need to have sufficient runs before tensorboard will start logging data. 

In [ ]:
# Train either the DNN or a SVM as classifier 
classifier_name = "dnn"

if classifier_name.lower() == "dnn":  
    dnn_model.train(train_set, lr_per_mb = lr_per_mb, mb_size = mb_size, eval_dataset=test_set) #, tensorboard_logdir=r"tensorboard"
    classifier = dnn_model
# You can uncomment the following lines and change the classifier_name to "svm" to enable svm classifier. 
# However, deployment of SVM classifier is not supported now.
# elif classifier_name.lower() == "svm":
#     learner = svm.LinearSVC(C=1.0, class_weight='balanced', verbose=0)
#     classifier = ScikitClassifier(dnn_model, learner = learner)
#     classifier.train(train_set)
else:
    raise Exception("Classifier unknown: " + classifier)   

In [ ]:
# Plot how the training and test accuracy increases during gradient descent. 
if classifier_name == "dnn":
    [train_accs, test_accs, epoch_numbers] = classifier.train_eval_accs
    plt.xlabel("Number of training epochs") 
    plt.ylabel("Classification accuracy") 
    train_plot = plt.plot(epoch_numbers, train_accs, 'r-', label = "Training accuracy")
    test_plot = plt.plot(epoch_numbers, test_accs, 'b-.', label = "Test accuracy")
    plt.legend()

### Step 6: Evaluation and Visualization

The evaluation module provides functionality to evaluate the performance of the trained model on an independent test dataset. Some of the evaluation metrics it computes are: accuracy (by default class-averaged), PR curve, ROC curve, area-under-curve, confusion metrix, etc.

In [ ]:
# Run the classifier on all test set images
ce = ClassificationEvaluation(classifier, test_set, minibatch_size = mb_size)

# Compute Accuracy and the confusion matrix
acc = ce.compute_accuracy()
print("Accuracy = {:2.2f}%".format(100*acc))
cm  = ce.compute_confusion_matrix()
print("Confusion matrix = \n{}".format(cm))

# Show PR curve, ROC curve, and confusion matrix
fig, ((ax1, ax2, ax3)) = plt.subplots(1,3)
fig.set_size_inches(18, 4)
graph_roc_curve(ce, ax=ax1)
graph_pr_curve(ce, ax=ax2)
graph_confusion_matrix(ce, ax=ax3)
plt.show()

In [ ]:
# Results viewer UI
labels = [l.name for l in dataset.labels] 
pred_scores = ce.scores #classification scores for all images and all classes
pred_labels = [labels[i] for i in np.argmax(pred_scores, axis=1)]

from ui_utils.ui_results_viewer import ResultsUI
results_ui = ResultsUI(test_set, Context.get_global_context(), pred_scores, pred_labels)
display(results_ui.ui)

In [ ]:
# Precision / recall curve UI
precisions, recalls, thresholds = ce.compute_precision_recall_curve() 
thresholds = list(thresholds)
thresholds.append(thresholds[-1])
from ui_utils.ui_precision_recall import PrecisionRecallUI
pr_ui = PrecisionRecallUI(100*precisions[::-1], 100*recalls[::-1], thresholds[::-1])
display(pr_ui.ui) 

### Step 7: Webservice Deployment


<b>Prerequisites:</b> 
Please the check the **Prerequisites** section of our deployment notebook to set up your deployment CLI. You only need to set it up once for all your deployments. More deployment related topics including IoT Edge deployment can be found in the deployment notebook.
       
<b>Deployment API:</b>

> **Examples:**
- ```deploy_obj = AMLDeployment(deployment_name=deployment_name, associated_DNNModel=dnn_model, aml_env="cluster")``` # create deployment object
- ```deploy_obj.deploy()``` # deploy web service
- ```deploy_obj.status()``` # get status of deployment
- ```deploy_obj.score_image(local_image_path_or_image_url)``` # score an image
- ```deploy_obj.delete()``` # delete the web service
- ```deploy_obj.build_docker_image()``` # build docker image without creating webservice
- ```AMLDeployment.list_deployment()``` # list existing deployment
- ```AMLDeployment.delete_if_service_exist(deployment_name)``` # delete if the service exists with the deployment name

<b>Deployment management with portal:</b>

You can go to [Azure portal](https://ms.portal.azure.com/) to track and manage your deployments. From Azure portal, find your Machine Learning Model Management account page (You can search for your model management account name). Then go to: the model management account page->Model Management->Services.

In [ ]:
# ##### OPTIONAL - Interactive CLI setup helper ###### 
# # Interactive CLI setup helper, including model management account and deployment environment.
# # If you haven't setup you CLI before or if you want to change you CLI settings, you can use this block to help you interactively.
# # UNCOMMENT THE FOLLOWING LINES IF YOU HAVE NOT CREATED OR SET THE MODEL MANAGEMENT ACCOUNT AND DEPLOYMENT ENVIRONMENT

# from azuremltkbase.deployment import CliSetup
# CliSetup().run()

In [ ]:
# # Optional. Persist you model on disk and reuse it later for deployment. 
# from cvtk import CNTKTLModel, Context
# import os
# save_model_path = os.path.join(Context.get_global_context().storage.persistent_path, "saved_classifier.model")
# # Save model to disk
# dnn_model.save(save_model_path)
# # Load model from disk
# dnn_model = CNTKTLModel.load(save_model_path)

In [ ]:
from cvtk.operationalization import AMLDeployment

# set deployment name
deployment_name = "icdeployment"

# Create deployment object
# It will use the current deployment environment (you can check it with CLI command "az ml env show").
deploy_obj = AMLDeployment(deployment_name=deployment_name, aml_env="cluster", associated_DNNModel=dnn_model, replicas=1)

# Alternatively, you can provide azure machine learning deployment cluster name (environment name) and resource group name
# to deploy your model. It will use the provided cluster to deploy. To do that, please uncomment the following lines to create 
# the deployment object.

# azureml_rscgroup = "<resource group>"
# cluster_name = "<cluster name>"
# deploy_obj = AMLDeployment(deployment_name=deployment_name, associated_DNNModel=dnn_model,
#                            aml_env="cluster", cluster_name=cluster_name, resource_group=azureml_rscgroup, replicas=1)

# Check if the deployment name exists, if yes remove it first. 
# Note: This will delete existing webservice with the same name. Do not delete the webservice unintentionally. 
if deploy_obj.is_existing_service():
    AMLDeployment.delete_if_service_exist(deployment_name)
    
# create the webservice
print("Deploying to Azure cluster...")
deploy_obj.deploy()
print("Deployment DONE")

### Step 8: Webservice comsumption

Once you created the webservice, you can score images with the deployed webservice. You have several options:

   - You can directly score the webservice with the deployment object with: deploy_obj.score_image(image_path_or_url) 
   - Or, you can use the Service endpoin url and Serivce key (None for local deployment) with: AMLDeployment.score_existing_service_with_image(image_path_or_url, service_endpoint_url, service_key=None)
   - Form your http requests directly to score the webservice endpoint (For advanced users).

#### Score with existing deployment object
```
deploy_obj.score_image(image_path_or_url)
```

In [ ]:
# Score with existing deployment object

# Score local image with file path
print("Score local image with file path")
image_path_or_url = test_set.images[0].storage_path
print("Image source:",image_path_or_url)
serialized_result_in_json = deploy_obj.score_image(image_path_or_url, image_resize_dims=[224,224])
print("serialized_result_in_json:", serialized_result_in_json)

# Score image url and remove image resizing
print("Score image url")
image_path_or_url = "https://cvtkdata.blob.core.windows.net/publicimages/microsoft_logo.jpg"
print("Image source:",image_path_or_url)
serialized_result_in_json = deploy_obj.score_image(image_path_or_url)
print("serialized_result_in_json:", serialized_result_in_json)

# Score image url with added paramters. Add softmax to score.
print("Score image url with added paramters. Add softmax to score")
from cvtk.utils.constants import ClassificationRESTApiParamters
image_path_or_url = "https://cvtkdata.blob.core.windows.net/publicimages/microsoft_logo.jpg"
print("Image source:",image_path_or_url)
serialized_result_in_json = deploy_obj.score_image(image_path_or_url, image_resize_dims=[224,224], parameters={ClassificationRESTApiParamters.ADD_SOFTMAX:True})
print("serialized_result_in_json:", serialized_result_in_json)

In [ ]:
# Time image scoring
import timeit

num_images = 10
for img_index, img_obj in enumerate(test_set.images[:num_images]):
    print("Calling API for image {} of {}: {}...".format(img_index, num_images, img_obj.name))
    tic = timeit.default_timer()
    return_json = deploy_obj.score_image(img_obj.storage_path, image_resize_dims=[224,224])
    print("   Time for API call: {:.2f} seconds".format(timeit.default_timer() - tic))
    print(return_json)

#### Score with service endpoint url and service key
`AMLDeployment.score_existing_service_with_image(image_path_or_url, service_endpoint_url, service_key=None)`

In [ ]:
# Import related classes and functions
from cvtk.operationalization import AMLDeployment

service_endpoint_url = "" # please replace with your own service url
service_key = "" # please replace with your own service key
# score local image with file path
image_path_or_url = test_set.images[0].storage_path
print("Image source:",image_path_or_url)
serialized_result_in_json = AMLDeployment.score_existing_service_with_image(image_path_or_url,service_endpoint_url, service_key = service_key)
print("serialized_result_in_json:", serialized_result_in_json)

# score image url
image_path_or_url = "https://cvtkdata.blob.core.windows.net/publicimages/microsoft_logo.jpg"
print("Image source:",image_path_or_url)
serialized_result_in_json = AMLDeployment.score_existing_service_with_image(image_path_or_url,service_endpoint_url, service_key = service_key, image_resize_dims=[224,224])
print("serialized_result_in_json:", serialized_result_in_json)

#### Score endpoint with http request directly
Following is some example code to form the http request directly in Python. You can do it in other programming languages.

In [ ]:
def score_image_with_http(image, service_endpoint_url, service_key=None, parameters={}):
    """Score local image with http request

    Args:
        image (str): Image file path
        service_endpoint_url(str): web service endpoint url
        service_key(str): Service key. None for local deployment.
        parameters (dict): Additional request paramters in dictionary. Default is {}.


    Returns:
        str: serialized result 
    """
    import requests
    from io import BytesIO
    import base64
    import json

    if service_key is None:
        headers = {'Content-Type': 'application/json'}
    else:
        headers = {'Content-Type': 'application/json',
                   "Authorization": ('Bearer ' + service_key)}
    payload = []
    encoded = None
    
    # Read image
    with open(image,'rb') as f:
        image_buffer = BytesIO(f.read()) ## Getting an image file represented as a BytesIO object
        
    # Convert your image to base64 string
    # image_in_base64 : "b'{base64}'"
    encoded = base64.b64encode(image_buffer.getvalue())
    image_request = {"image_in_base64": "{0}".format(encoded), "parameters": parameters}
    payload.append(image_request)
    body = json.dumps(payload)
    r = requests.post(service_endpoint_url, data=body, headers=headers)
    try:
        result = json.loads(r.text)
        json.loads(result[0])
    except:
        raise ValueError("Incorrect output format. Result cant not be parsed: " + r.text)
    return result[0]

# Test with images
image = test_set.images[0].storage_path # A local image file
score_image_with_http(image, service_endpoint_url, service_key) # Local scoring the service_key is None

### Parse serialized result from webservice
The result from the webserice is in json string. You can parse it the with different DNN model classes

In [ ]:
image_path_or_url = test_set.images[0].storage_path
print("Image source:",image_path_or_url)
serialized_result_in_json = deploy_obj.score_image(image_path_or_url, image_resize_dims=[224,224])
print("serialized_result_in_json:", serialized_result_in_json)

In [ ]:
# Parse result from json string
import numpy as np
parsed_result = CNTKTLModel.parse_serialized_result(serialized_result_in_json)
print("Parsed result:", parsed_result)
# Map result to image class
class_index = np.argmax(np.array(parsed_result))
print("Class index:", class_index)
dnn_model.class_map
print("Class label:", dnn_model.class_map[class_index])


# APPENDIX 

# (A) Bing image download

The dataset object provides functionality to download images using the [Bing Image Search API](https://azure.microsoft.com/en-us/services/cognitive-services/bing-image-search-api/). Two types of search queries are supported: (i) regular text queries; and (ii) image URL queries. These queries need to be provided inside a json-encoded text file. In addition to the search queries, also their class label has to be specified (see this [sample json file](./misc/sample.json)). Furthermore, a Context object needs to be created explicitly and contain the Bing Image Search API key which requires a Bing Image Search API subscription. 



# (B) How to improve accuracy
The Computer Vision toolkit (CVTK) is shown to give good results for a wide variety of datasets. However, as is true for most machine learning projects, getting the best possible results for a new dataset requires careful parameter tuning, as well as evaluating different design decisions. The following sections provide guidance on how to improve accuracy on a given dataset, ie. what parameters are most promising to optimize first, what values for these parameters one should try, and what pitfalls to avoid.

Generally speaking, training Deep Learning models comes with a trade-off between training time versus model accuracy. The CVTK has pre-set default parameters (see 1st row in the table below) which focus on fast training speed while typically producing high accuracy models. This accuracy can often be improved further using e.g. higher image resolution or deeper models, however at the cost of increasing training time by a factor of 10x or more.

Our recommendation is to first work with the default parameters, train a model, inspect the results, correct ground truth annotations as needed, and only then try parameters which slow down training time (see table below suggested parameter values). An understanding of these parameters while technically not necessary is however recommended.


### Best-practice / Pitfalls / Tips:
* Data quality: the training and test sets should be of high quality. That is, the images are annotated correctly, ambiguous images removed (for example where it is unclear to a human eye if the image shows a tennis ball or a lemon), and the attributes are mutually exclusive (that is, each image belongs to exactly one attribute).
* Before refining the DNN, a SVM classifier should be trained using a pre-trained and fixed DNN as featurizer. This is supported in CVTK and does not require long to train since the DNN itself is not modified. Even this simple approach often achieves good accuracies and hence represents a strong baseline. The next step is then to refine the DNN which should give better accuracy.
* If the object-of-interest is small in the image then Image classification approaches are known to not work well. In such cases, consider using an object detection approach such as CVTK's Faster R-CNN based on Tensorflow.
* The more training data the better. As a rule-of-thumb, one should have at least 100 examples for each class, ie. 100 images for "dog", 100 images for "cat", etc. Training a model with less images is possible but might not produce good results.
* The training images need to reside locally on the machine with the GPU, and be on an SSD drive (not a HDD). Otherwise, latency from image reading can drastically reduce training speed (by even a factor of 100x).


### Parameters to optimize:
Finding optimal values for these parameters is important and can often improve accuracy significantly:
* Learning rate (`lr_per_mb`): The arguably most important parameter to get right. If the accuracy on the training set after DNN refinement is above ~5% then most likely the learning rate is either too high, or the number of training epochs too low. Especially with small datasets, the DNN tends to over-fit on the training data, however in practice this still leads to good models on the test set. We typically use 15 epochs where the inital learning rate is reduced twice; training using more epochs can in some cases improve performance.
* Input resolution (`image_dims`): The default image resolution is 224x224 pixels. Using higher image resolution of, for example, 500x500 pixels or 1000x1000 pixels can significantly improve accuracy but slows down DNN refinement. Note that CVTK expects the input resolution to be a tuple of (color-channels, image-width, image-height), e.g. (3, 224, 224), where the number of color channels has to be set to 3 (the Red-Green-Blue bands).
* Model architecture(`base_model_name`): Try using deeper DNNs such as ResNet-34 or ResNet-50 instead of the default ResNet-18 model. The Resnet-50 model is not only deeper, but its output of the penultimate layer is of size 2048 floats (vs. 512 floats of the ResNet-18 and ResNet-34 models). This increased dimensionality can be especially beneficial when keeping the DNN fixed and instead training an SVM classifier.
* Minibatch size (`mb_size`): High minibatch sizes will lead to faster training time however at the expense of an increased DNN memory consumption. Hence, when selecting deeper models (E.g. ResNet-50 versus ResNet-18) and/or higher image resolution (500\*500 pixels versus 224\*224 pixels), one typically has to reduce the minibatch size to avoid out-of-memory errors. When changing the minibatch size, often also the learning rate needs to be adjusted as can be seen in the table below.
* Drop-out rate (`dropout_rate`) and L2-regularizer (`l2_reg_weight`): DNN over-fitting can be reduced by using a dropout rate of 0.5 (default is 0.5 in CVTK) or more, and by increasing the regularizer weight (default is 0.0005 in CVTK). Note though that especially with small datasets DNN over-fitting is hard and often impossible to avoid.


### Parameter explanation:
- Learning rate: step size used during gradient descent learning. If set too low then the model will take many epochs to train, if set too high then the model will not converge to a good solution. Note that typically a schedule is used where the learning rate is reduced after a certain number of epochs. E.g. the learning rate schedule `[0.05]*7 + [0.005]*7 + [0.0005]` corresponds to using an initial learning rate of 0.05 for the first 7 epochs, followed by a 10x reduced learning rate of 0.005 for another 7 epochs, and finally fine-tuning the model for a single epoch with a 100x reduced learning rate of 0.0005.
- Minibatch size: GPU's can process multiple images in parallel to speed up computation. These parallel processed images are also referred as a minibatch. The higher the minibatch size the faster training will be, however at the expense of a significantly increased DNN memory consumption.

### Suggested parameters values:
The table below provides different parameter sets which were shown to produce high accuracy models on a wide variety of image classification tasks. The optimal parameters depend on the specific dataset and on the exact GPU used, hence the table should be seen as a guideline only. After trying these parameters, consider also image resolutions of more than 500x500 pixels, or deeper models such as Resnet-101 or Resnet-152.

The first row in the table corresponds to the default parameters which are set inside CVTK. All other rows take longer to train (indicated in the first column) however at the benefit of increased accuracy (see the second column for the average accuracy over three internal datasets). For example, the parameters in the last row take 5-15x longer to train, however resulted in significantly increased (averaged) accuracy on three internal test sets from 82.6% to 92.8%.

Deeper models and higher input resolution take up more DNN memory, and hence the minibatch size needs to be reduced with increased model complexity to avoid out-of-memory-errors. As can be seen in the table below, we found it beneficial to decrease the learning rate by a factor of two whenever decreasing the minibatch size by the same multiplier. Note that the minibatch size might need to get reduced further on GPUs with smaller amounts of memory.

| Training time (rough estimate) | Example accuracy | Minibatch size (`mb_size`) | Learning rate (`lr_per_mb`) | Image resolution (`image_dims`) | DNN architecture (`base_model_name`) |
|------------- |:-------------:|:-------------:|:-----:|:-----:|:---:|
| 1x (reference) | 82.6% | 32 | [0.05]\*7  + [0.005]\*7  + [0.0005]  | (3, 224, 224) | ResNet18_ImageNet_CNTK |
| 2-5x    | 90.2% | 16 | [0.025]\*7 + [0.0025]\*7 + [0.00025] | (3, 500, 500) | ResNet18_ImageNet_CNTK |
| 2-5x    | 87.5% | 16 | [0.025]\*7 + [0.0025]\*7 + [0.00025] | (3, 224, 224) | ResNet50_ImageNet_CNTK |
| 5-15x        | 92.8% |  8 | [0.01]\*7  + [0.001]\*7  + [0.0001]  | (3, 500, 500) | ResNet50_ImageNet_CNTK |


© 2018 Microsoft. All rights reserved. 